In [0]:
import theano
import theano.tensor as T
import numpy as np

# $A^k$

In [0]:
k = T.iscalar('k')
A = T.vector( 'A')

results, updates = theano.scan(lambda priopri_result, A : priopri_result * A,
             non_sequences = A, outputs_info=T.ones_like(A), n_steps = k)

In [0]:
fn_Ak = theano.function([A, k], results)

In [0]:
print(fn_Ak(range(10), 2))

[[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9.]
 [ 0.  1.  4.  9. 16. 25. 36. 49. 64. 81.]]


In [0]:
print(fn_Ak(range(10), 3))

[[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.]
 [  0.   1.   4.   9.  16.  25.  36.  49.  64.  81.]
 [  0.   1.   8.  27.  64. 125. 216. 343. 512. 729.]]


In [0]:
print(fn_Ak(range(0, 10, 2), 2))

[[ 0.  2.  4.  6.  8.]
 [ 0.  4. 16. 36. 64.]]


# $A^k$

In [0]:
k = T.iscalar('k')
A = T.iscalar('A')

results, updates = theano.scan(lambda priopri_result, A: priopri_result*A,
                               non_sequences = A,
                               outputs_info = np.asarray(1, A.dtype), 
                               n_steps = k)

In [0]:
fn_Ak_scan = theano.function([A, k], results)

In [0]:
print(fn_Ak_scan(3, 3))

[ 3  9 27]


# $f = 1 + 2 + 3 + ... + n$

In [0]:
v = T.vector('v')

results, updates = theano.scan(fn=lambda v, priori_result: v + priori_result, 
                               sequences=v, 
                               outputs_info=np.asarray(0, v.dtype))

In [0]:
sum_scan = theano.function([v], results)

In [0]:
sum_scan([1, 2, 3])

array([1., 3., 6.])

# $f = 1 + 2 + 3 + ... + n$

In [0]:
x = T.scalar('x')
seq=T.arange(x)

results, updates = theano.scan(fn = lambda x, priori_result: x + priori_result,
                              sequences=seq,
                              outputs_info=np.asarray(0, x.dtype))

In [0]:
scalar_sum_scan = theano.function([x], results)

In [0]:
scalar_sum_scan(10)

array([ 0.,  1.,  3.,  6., 10., 15., 21., 28., 36., 45.])

# $f = 1 + 2 + 3 + ... + n$

In [0]:
x = T.imatrix('x')
  
results, updates = theano.scan(fn = lambda x, priori_result: x + priori_result,
                              sequences=x,
                              outputs_info=T.zeros_like(x))

In [0]:
matrix_scan = theano.function([x], results)

In [0]:
print(matrix_scan([[1, 2], [3, 4]]))

[[[1 2]
  [1 2]]

 [[4 6]
  [4 6]]]


In [0]:
for e in [[1, 2], [3, 4]]:
  print(e)
  print("##")

[1, 2]
##
[3, 4]
##


# $f = 1 + 2 + 3 + ... + n$

In [0]:
x = T.imatrix('x')

results, updates = theano.scan(fn = lambda x, priori_result: x + priori_result,
                              sequences=theano.tensor.join(0, x),
                              outputs_info=T.zeros_like(x))

In [0]:
matrix_scan = theano.function([x], results)

In [0]:
print(matrix_scan([[1, 2], [3, 4]]))

[[[1 2]
  [1 2]]

 [[4 6]
  [4 6]]]


# $\sum_{n=0}^Na_nX^n$

In [0]:
x = T.scalar('x')

A = T.vector('A')

N = T.ivector('N')

results, updates = theano.scan(fn = lambda A, N, x: A*(x**N),
                               sequences=[A, N],
                               non_sequences=x)

In [0]:
result = results.sum()

polynomial = theano.function([x, A, N], result)

In [0]:
print(polynomial(10, 
                 [1, 3, 2],
                 [0, 2, 3]))

2301.0


# $\sum_{n=0}^Na_nX^n$

In [0]:
x = T.iscalar('x')

A = T.ivector('A')

N = T.ivector('N')

results, updates = theano.scan(fn = lambda A, N, priopri_result, x: priopri_result+A*(x**N),
                               sequences=[A, N],
                               non_sequences=x, 
                               outputs_info=np.asarray(0, x.dtype))

In [0]:
polynomial_scan = theano.function([x, A, N], results)

In [0]:
print(polynomial_scan(10, 
                 [1, 3, 2],
                 [0, 2, 3]))

[   1  301 2301]


In [0]:
print(np.array(10))

10


# $f = 1 + 2 + 3 + ... + n$
共享变量

In [0]:
n = theano.shared(np.asarray(0))
k = T.iscalar("k")

# 这里 lambda 的返回值是一个 dict，因此这个值会被传入 updates 中
_, updates = theano.scan(fn = lambda n: {n:n+1},
                         non_sequences = n,
                         n_steps = k)

In [0]:
counter = theano.function(inputs = [k], 
                          outputs = [],
                          updates = updates)

In [0]:
print(n.get_value())
counter(10)
print(n.get_value())
counter(10)
print(n.get_value())

0
10
20


# until

In [0]:
max_value = T.scalar()

results, _ = theano.scan(fn = lambda v_pre, max_v: (v_pre * 2, theano.scan_module.until(v_pre * 2 > max_v)), 
                         outputs_info = T.constant(1.),
                         non_sequences = max_value,
                         n_steps = 1000)

In [0]:
# 注意，这里不能取 results 的全部
# 例如在输入值为 40 时，最后的输出可以看成 (64, False)
# scan 发现停止条件满足，停止循环，但是不影响 64 被输出到 results 中，因此要将 64 去掉
power_of_2 = theano.function(inputs = [max_value], outputs = results[:-1])

In [0]:
print(power_of_2(40))

[ 2.  4.  8. 16. 32.]


In [0]:
max_value = T.scalar()

results, _ = theano.scan(fn = lambda v_pre, max_v: (v_pre * 2, theano.scan_module.until(v_pre * 2 > max_v)), 
                         outputs_info = T.constant(1.),
                         non_sequences = max_value,
                         n_steps = 1000)

In [0]:
power_of_2_all = theano.function(inputs = [max_value], outputs = results)

In [0]:
print(power_of_2_all(40))

[ 2.  4.  8. 16. 32. 64.]
